In [2]:
import pandas as pd
import pyodbc
from os.path import join, normpath

downloads = 'C:/Users/balob/Downloads/DATA'

file_name = 'NEW_TARIFFS.csv'
file_name_res = 'NEW_TARIFFS_result.csv'

In [10]:
#Из базы данных OCS выгружаем TADIG и список дискаунтеров
 
#Connect to OCSDBREP1 (BSS)
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')

sql_srt=\
'''
SELECT * FROM RDB_TADIG_CODES
'''
df_tadig = pd.read_sql_query(sql_srt, cnxn)
df_tadig['TADIG_CODE_ID']=df_tadig['TADIG_CODE_ID'].astype('int')
df_tadig['NETWORK_ID']=df_tadig['NETWORK_ID'].astype('int')
display(df_tadig.head())

sql_srt=\
'''
SELECT * FROM ROAMING_PLAN_RULES WHERE roaming_plan_id=323 AND [end_date] IS NULL AND [is_discounted]=1
'''
df_disc = pd.read_sql_query(sql_srt, cnxn)
display(df_disc.head())

cnxn.close()

,TADIG_CODE_ID,NETWORK_ID,TADIG_CODE
0,987,4,AFGAW
1,988,5,AFGAR
2,989,6,AFG55
3,990,7,AFGTD
4,991,9,ALBAM


,ROAMING_PLAN_RULE_ID,ROAMING_PLAN_ID,NETWORK_ID,SPONSOR_ID,MO_CALL_RATE,MT_CALL_RATE,MO_SMS_RATE,MT_SMS_RATE,DATA_RATE,CURRENCY_ID,START_DATE,END_DATE,is_discounted,HIDDEN
0,22540.0,323.0,20.0,2.0,1.920,0.0,0.192,0.0,0.19200,1.0,2019-04-10,None,True,True
1,22541.0,323.0,23.0,2.0,1.920,0.0,0.192,0.0,0.19200,1.0,2019-04-10,None,True,True
2,22542.0,323.0,55.0,2.0,1.008,0.0,0.168,0.0,0.02688,1.0,2019-04-10,None,True,True
3,22543.0,323.0,70.0,2.0,1.920,0.0,0.192,0.0,0.19200,1.0,2019-04-10,None,True,True
4,22544.0,323.0,122.0,2.0,1.920,0.0,0.192,0.0,0.19200,1.0,2019-04-10,None,True,True


In [11]:
#Загружаем данные из csv файла с тарифами
#Пример файла NEW_TARIFFS.csv

df=pd.read_csv(join(downloads, file_name),sep=';')
df.head(3)

,Country,Partner Name,TADIG,MO_CALL_RATE,MT_CALL_RATE,MO_SMS_RATE,DATA_RATE,OutgoingVoiceIncrement_SEC,DataIncrement_KB
0,Albania,Vodafone,ALBVF,20,20,6.0,3.0,NaN,NaN
1,Anguilla,Cable and Wireless,AIACW,20,20,6.0,3.0,NaN,NaN
2,Antigua and Barbuda,C & W,ATGCW,20,20,6.0,3.0,NaN,NaN


In [12]:
#Объединяем тарифы с TADIG

df1=pd.merge(df,df_tadig,left_on='TADIG',right_on='TADIG_CODE',how='left')
df1.head(3)

,Country,Partner Name,TADIG,MO_CALL_RATE,MT_CALL_RATE,MO_SMS_RATE,DATA_RATE,OutgoingVoiceIncrement_SEC,DataIncrement_KB,TADIG_CODE_ID,NETWORK_ID,TADIG_CODE
0,Albania,Vodafone,ALBVF,20,20,6.0,3.0,NaN,NaN,994,12,ALBVF
1,Anguilla,Cable and Wireless,AIACW,20,20,6.0,3.0,NaN,NaN,1000,20,AIACW
2,Antigua and Barbuda,C & W,ATGCW,20,20,6.0,3.0,NaN,NaN,1002,23,ATGCW


In [13]:
#Проверяем, что ничего не пропало

print(df.count(),'\n')
print(df1.count())

df1[df1.TADIG_CODE_ID.isnull()]

# df1[df1.TADIG=='FINTA']

Country                       278
Partner Name                  278
TADIG                         278
MO_CALL_RATE                  278
MT_CALL_RATE                  278
MO_SMS_RATE                   278
DATA_RATE                     278
OutgoingVoiceIncrement_SEC      0
DataIncrement_KB                0
dtype: int64 

Country                       278
Partner Name                  278
TADIG                         278
MO_CALL_RATE                  278
MT_CALL_RATE                  278
MO_SMS_RATE                   278
DATA_RATE                     278
OutgoingVoiceIncrement_SEC      0
DataIncrement_KB                0
TADIG_CODE_ID                 278
NETWORK_ID                    278
TADIG_CODE                    278
dtype: int64


,Country,Partner Name,TADIG,MO_CALL_RATE,MT_CALL_RATE,MO_SMS_RATE,DATA_RATE,OutgoingVoiceIncrement_SEC,DataIncrement_KB,TADIG_CODE_ID,NETWORK_ID,TADIG_CODE


In [15]:
# Проверить, все ли дискаунтеры. Для сверки берем один из тарифных планов.
# Данный шаг нужен для плано сторонних компаний, создаваемых на основе предоставленных ими данных. Например, Махком.

df2=pd.merge(df1,df_disc[['NETWORK_ID','is_discounted']],on='NETWORK_ID',how='left')
display(df2.head(3))
df2[df2.is_discounted.isnull()]

,Country,Partner Name,TADIG,MO_CALL_RATE,MT_CALL_RATE,MO_SMS_RATE,DATA_RATE,OutgoingVoiceIncrement_SEC,DataIncrement_KB,TADIG_CODE_ID,NETWORK_ID,TADIG_CODE,is_discounted
0,Albania,Vodafone,ALBVF,20,20,6.0,3.0,NaN,NaN,994,12,ALBVF,True
1,Anguilla,Cable and Wireless,AIACW,20,20,6.0,3.0,NaN,NaN,1000,20,AIACW,True
2,Anguilla,Cable and Wireless,AIACW,20,20,6.0,3.0,NaN,NaN,1000,20,AIACW,True


,Country,Partner Name,TADIG,MO_CALL_RATE,MT_CALL_RATE,MO_SMS_RATE,DATA_RATE,OutgoingVoiceIncrement_SEC,DataIncrement_KB,TADIG_CODE_ID,NETWORK_ID,TADIG_CODE,is_discounted


In [16]:
#Добавляем поля для удобства выгрузки
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# !!! Поменять 'ROAMING_PLAN_ID', 'CURRENCY_ID', 'SPONSOR_ID' и 'START_DATE'
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

df2['ROAMING_PLAN_RULE_ID']=''
df2['ROAMING_PLAN_ID']=308
df2['SPONSOR_ID']='NULL'
df2['MT_SMS_RATE']=0
df2['CURRENCY_ID']=1 #USD
df2['START_DATE']='2019-06-01 00:00:00'
df2['END_DATE']='NULL'
#df2['is_discounted']=1
df2['is_discounted']=df2.is_discounted.fillna(value = 0).astype('int64')
df2.head()

,Country,Partner Name,TADIG,MO_CALL_RATE,MT_CALL_RATE,MO_SMS_RATE,DATA_RATE,OutgoingVoiceIncrement_SEC,DataIncrement_KB,TADIG_CODE_ID,NETWORK_ID,TADIG_CODE,is_discounted,ROAMING_PLAN_RULE_ID,ROAMING_PLAN_ID,SPONSOR_ID,MT_SMS_RATE,CURRENCY_ID,START_DATE,END_DATE
0,Albania,Vodafone,ALBVF,20,20,6.0,3.0,NaN,NaN,994,12,ALBVF,1,,308,NULL,0,1,2019-06-01 00:00:00,NULL
1,Anguilla,Cable and Wireless,AIACW,20,20,6.0,3.0,NaN,NaN,1000,20,AIACW,1,,308,NULL,0,1,2019-06-01 00:00:00,NULL
2,Anguilla,Cable and Wireless,AIACW,20,20,6.0,3.0,NaN,NaN,1000,20,AIACW,1,,308,NULL,0,1,2019-06-01 00:00:00,NULL
3,Antigua and Barbuda,C & W,ATGCW,20,20,6.0,3.0,NaN,NaN,1002,23,ATGCW,1,,308,NULL,0,1,2019-06-01 00:00:00,NULL
4,Antigua and Barbuda,C & W,ATGCW,20,20,6.0,3.0,NaN,NaN,1002,23,ATGCW,1,,308,NULL,0,1,2019-06-01 00:00:00,NULL


In [17]:
#Выгружаем без дубликатов в csv

df2[['ROAMING_PLAN_RULE_ID',
'ROAMING_PLAN_ID',
'NETWORK_ID',
'SPONSOR_ID',
'MO_CALL_RATE',
'MT_CALL_RATE',
'MO_SMS_RATE',
'MT_SMS_RATE',
'DATA_RATE',
'CURRENCY_ID',
'START_DATE',
'END_DATE',
'is_discounted',
'OutgoingVoiceIncrement_SEC',
'DataIncrement_KB']].drop_duplicates().to_csv(join(downloads, file_name_res),index=False)

-----------------------------------------------------------------------------------------
Далее используем INSERT_ROAMING_PLAN_RULES_ХХХХХХ.xlsx